<a href="https://colab.research.google.com/github/kylechanpols/sid/blob/main/training_demo_technical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using the U-Net to measure local economic development

Kyle Chan (UNC-Chapel Hill, kylechan@unc.edu)
(with some code adapted from Yann Le Guilly)

Welcome to the demo page for my project to measure local economic development using satellite imagery! In this Google Colab workbook I will explain every step taken to construct the local development index using satellite images through the U-Net architecture. As of 9/4, the model is trained on around 3400 images. When tested on a test set of around 430 images, it has achieved a 91% test set accuracy and a mean intersection-over-union score of around 90.5%.

## Motivation

In the literature of subnational politics, we don't really have very micro-level measure of important qualities such as political decentralization, economic development, etc at the local level. This is very unfortunate because our theorization tends to happen on a very local scale, but then the data we have are still very aggregated.

### Measuring Local Economic Development

On the measurement of local economic development we run into the same issue. Economists have used two approaches two get around this issue: One is with luminosity score, which utilizes a satellite to capture images of a local area. The images are taken at night, which they theorize that more developed areas are more likely to have more nightlights than underdeveloped areas. This approach has so far been the state-of-the-art for many years in that literature. However, it does have several limitations:

- Clouds can obscure the luminosity on the ground level. Some correction can be made by taking multiple images at different times and exclude cloudy ones. But if the location is prone to clouds often then the adjustment may not work.
- Nightlights may not portray an accurate picture of the ground truth due to policy constraints, e.g. some Nordic countries turn off street lights at night to protect birds.

Another approach is to abstract the GDP level using an assortiment of weighting methods. This approach is also known as the "Gross Cell Product" (GCP) measure. However, it still has its fair share of limitations:

- The GCP measure is computed on other various measures that might have severe inaccuracy, particularly when measuring developing countries where the data quality can be sometimes questionable.
- The GCP is also subject to problems of missing data, particularly when used to measure developing countries.

Thus an alternative approach is required if we were to go even lower to capture local economic development.


## Measuring Local Economic Development with Satellite Images

One way I argue that we can measure local economic development is by using satellite images. Consider the case that we need to measure the level of local economic development of a city. We can look from above and count how many buildings are there in a city. Most people would agree that cities are more developed than rural areas, and thus all we need is to reframe the question and think about how much buildings/roads are there in an image vs. what portion of the image is a forest or other types of natural terrain untouched by human economic activity.

Consider these two images:

<img src="https://drive.google.com/uc?id=1f03r6xFe4S27-oIjdUIzcZlowWayuGFs" alt="Rural" width="256"/>
<img src="https://drive.google.com/uc?id=12NYztvcf0-WqEKkDSuhys19CF4BYJo7d" alt="Urban" width="256"/>

The left hand side is a snapshot of a countryside. When compared to the image to the right, a urban suburb, we can see that they contain drastically different information:
- Overall tone: The rural image has a lot more forest and farmland than the urban image. In other words, this means that the overall Green channel of the image to the left will have higher values than the urban image.
- Presence of White/Gray Classes: The rural image has fewer buildings than the urban image. Buildings usually have gray or bright color roofs. In the urban image, that is much more common than the rural image.
- Density of items: Items in the rural image are less densely packed than the urban image. In the urban image, the buildings are all closely stacked next to each oither. In other words, there are more lines separating buildings in the urban image. Shapes of buildings in the urban image also tend to be more rectangular than the fuzzly shaped farmlands and forests in the rural image.

These are important **features** that the deep learning model can learn from. With these features the model should be able to differentiate a rural, less developed image from an urban, more developed image.

# The Model

## Convolutional Neural Network

For images, it is very common to use convolutional neural networks (CNN/ ConvNets). To be brief, ConvNets parse an image through an operation called "convolution", and learn model weights in each convolution.

A **convolution** can be think of a way to abstract or morph an image such that the model can focus on particular parts of an image. These instructions on where to focus in an image are called **filters**. Filters can be of any size, and contain values that emphasize which part of an image gets more weight, and which part gets less. For example, consider the following (3,3) filter:

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-0lax{text-align:left;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-0lax">1</th>
    <th class="tg-0lax">0</th>
    <th class="tg-0lax">-1</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0lax">1</td>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">-1</td>
  </tr>
  <tr>
    <td class="tg-0lax">1</td>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">-1</td>
  </tr>
</tbody>
</table>

Let 1 denote a bright pixel and -1 denote a dark pixel. Then what the filter means is that we are reading in a light edge to the right, where the third column contains a column vector of darker pixels.

A **convolution** uses a sliding window algorithm to go through each part of the subnmatrix within an image, compute an element-wise product between the submatrix and the filter, and then sum the contents of the matrix into a scalar in the output. 

The **padding** of a convolution denotes how far ahead should the sliding window algorithm "look ahead" for submatrices in the image. By default, most deep learning frameworks would have padding=1, which means that the sliding window will shift right by one column (or shift down by one row after the columns have been exhausted) and compute convolutions from those submatrices.

## Example:
Consider the following $3*3*1$ (grayscale) image:

<table class="tg">
<thead>
  <tr>
    <th class="tg-0lax">3</th>
    <th class="tg-0lax">0</th>
    <th class="tg-0lax">1</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0lax">1</td>
    <td class="tg-0lax">5</td>
    <td class="tg-0lax">8</td>
  </tr>
  <tr>
    <td class="tg-0lax">2</td>
    <td class="tg-0lax">7</td>
    <td class="tg-0lax">2<br></td>
  </tr>
</tbody>
</table>

Say we apply a $3*3$ filter once:

<table class="tg">
<thead>
  <tr>
    <th class="tg-0lax">1</th>
    <th class="tg-0lax">0</th>
    <th class="tg-0lax">-1</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0lax">1</td>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">-1</td>
  </tr>
  <tr>
    <td class="tg-0lax">1</td>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">-1</td>
  </tr>
</tbody>
</table>

The outcome of this covolution is -5.

### Explanation

We compute an element-wise product by multiplying cell to cell from the image to the filter. Therefore, the only submatrix in this convolution is:

<table class="tg">
<thead>
  <tr>
    <th class="tg-0lax">3</th>
    <th class="tg-0lax">0</th>
    <th class="tg-0lax">-1</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0lax">1</td>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">-8</td>
  </tr>
  <tr>
    <td class="tg-0lax">2</td>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">-2</td>
  </tr>
</tbody>
</table>

We compute the sum of all elements in this submatrix: 3+0-1+1+0-8+2+0-2 = -5.

For a more complicated example, consider the following $4*4*1$ grayscale image:

<table class="tg">
<thead>
  <tr>
    <th class="tg-0lax">1</th>
    <th class="tg-0lax">2</th>
    <th class="tg-0lax">3</th>
    <th class="tg-0lax">4</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0lax">4</td>
    <td class="tg-0lax">3</td>
    <td class="tg-0lax">2</td>
    <td class="tg-0lax">1</td>
  </tr>
  <tr>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">0</td>
  </tr>
  <tr>
    <td class="tg-0lax">5</td>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">5</td>
  </tr>
</tbody>
</table>

Applied to the same $3*3$ filter above once, with padding=1. You should get a small $2*2$ output image:

<table class="tg">
<thead>
  <tr>
    <th class="tg-0lax">0</th>
    <th class="tg-0lax">0</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0lax">9</td>
    <td class="tg-0lax">-3</td>
  </tr>
</tbody>
</table>

### Explanation:
Use the sliding window algorithm to find 4 submatrices: they should be the following:
<table style="float: left; width:25%">
<thead>
  <tr>
    <th class="tg-0lax">1</th>
    <th class="tg-0lax">2</th>
    <th class="tg-0lax">3</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0lax">4</td>
    <td class="tg-0lax">3</td>
    <td class="tg-0lax">2</td>
  </tr>
  <tr>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">0</td>
  </tr>
</tbody>
</table>
<table style="float: left; width:25%">
<thead>
  <tr>
    <th class="tg-0lax">2</th>
    <th class="tg-0lax">3</th>
    <th class="tg-0lax">4</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0lax">3</td>
    <td class="tg-0lax">2</td>
    <td class="tg-0lax">1</td>
  </tr>
  <tr>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">0</td>
  </tr>
</tbody>
</table>
<table style="float: left; width:25%">
<thead>
  <tr>
    <th class="tg-0lax">4</th>
    <th class="tg-0lax">3</th>
    <th class="tg-0lax">2</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">0</td>
  </tr>
  <tr>
    <td class="tg-0lax">5</td>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">0</td>
  </tr>
</tbody>
</table>
<table style="float: left; width:25%">
<thead>
  <tr>
    <th class="tg-0lax">3</th>
    <th class="tg-0lax">2</th>
    <th class="tg-0lax">1</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">0</td>
  </tr>
  <tr>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">5</td>
  </tr>
</tbody>
</table>

Compute the element-wise product for each of the submatrices with the filter, then sum:
$$1+0-3+4+0-2+0+0+0 = 0$$
$$2+0-4+3+0-1+0+0+0 = 0$$
$$4+0-2+0+0+0+5+0+0 = 9$$
$$3+0-1+0+0+0+0+0-5 = -3$$

And thus the output is $2*2$:
<table class="tg">
<thead>
  <tr>
    <th class="tg-0lax">0</th>
    <th class="tg-0lax">0</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0lax">9</td>
    <td class="tg-0lax">-3</td>
  </tr>
</tbody>
</table>

In ConvNets, there will be convolution steps to abstract or morph the image as such. There are model weights that are associated with each element in the convolution output. Depending on the architecture, there could be an activation function following the convolution.




## Semantic Segmentation

The way that we can appraoch this machine learning problem is by the technique of semantic segmentation. Semantic segmentation means that we are assiging classes to each pixel of the image. The model then learns and predicts at the pixel level of the image. The segmentation task of finding different areas in an image is known as **semantic segmentation**. The segmentation task can also be applied to object detection, and that variant is called **instance segmentation**.

An image where each pixel is differentiated into different classes are called a **segmentation mask**. For example, consider the urban image example from before. We can highlight all buildings and roads and label them as "1"s, and everything else as "0"s. The resultant mask (by scaling everything by 255 such that it is a proper grayscale image) would look like the one to the right:

<img src="https://drive.google.com/uc?id=12NYztvcf0-WqEKkDSuhys19CF4BYJo7d" alt="Urban" width="256"/>
<img src="https://drive.google.com/uc?id=1VMBxH81uwqydFm4BS1yOdftRTwy00E4R" alt="Rural" width="256"/>

The task for the model, then is to learn from features in the source image to the left and find out how the features are related to the outcome in the segmentation mask.


## The U-Net architecture

In this paper I used the U-Net Architecture (Ronneberger, Fischer & Brox, 2015).The U-Net architecture has an equal number of encoding layers and decoding layers (and thus the architecture looks like a big U). The architecture makes extensive use of skip connection (i.e. caching output from the encoding layers and stack them ontop of the decoding layers).

The main advantage of a model that makes extensive use of skip connections is the runtime. With skip connection we skipped the calculations of many of the convolutions computed in the encoding stage. In addition, skip connection has another advantage of avoiding the problem of vanishing gradients (i.e. gradients for features become so small that it's essentially 0).

When visualized, the U-Net architecture looks like the following:

![](https://drive.google.com/uc?id=1lrkTBa7mpvNoDxLavD45W3drDGDaMSIj)

## Loss

The loss function used here is the binary cross-entrophy:
$$L(\theta) = \frac{1}{n}\sum^n_{i=1} \ [y_i log(p_i)+(1-y_i)log(1-p_i)]$$

where $p_i \in (0,1)$ denotes the probability of $y_i$ is exactly of class 1. $y_i$ is the class label, where 0 denotes a non-building pixel and 1 denotes a building pixel.

## The Adam Optimizer

The Adaptive Moment Estimation (Adam) optimizer is used for this problem. Adam is preferred over other optimizers because it has the best performance out of all existing optimizers in the market. It combines the idea of the exponentially weighted average to incorporate the notion of momentum, along with Root Mean Square Propagation (RMSProp) - so it's a hybrid of two optimization techniques in the market. In the following test we see that the training cost is the lowest with Adam:

![](https://machinelearningmastery.com/wp-content/uploads/2017/05/Comparison-of-Adam-to-Other-Optimization-Algorithms-Training-a-Multilayer-Perceptron.png)

It is also a highly versatile optimizer, with up to 4 hyperparameters available for tuning:
- $\alpha$ - learning rate
- $\beta_1$ - can be understood as the rate controlling the velocity for the gradients
- $\beta_2$ - can be understood as the rate controlling the velocity for the squared gradients (part of the RMSProp)
- $\epsilon$ - adjustment to the RMSProp correction

### Learning Rate Decay

In this model I have also incorporate learning rate decay with rate = 0.9. Recall that learning rate decay means that the learning rate $\alpha$:
$$\alpha = \frac{1}{1+ decayRate*epochNr.}$$

Depending on how large `decayRate` is, we can control how quickly or slowly does the model adapt to a solution. A large `decayRate` implies that we will spread out the learning rate a bit more, thus slowing down learning in the initial epochs.

## Metrics

Two metrics are used in this problem:
- Accuracy
- Mean Intersection-over-union (IoU)

### Accuracy

As the name suggests, accuracy shows how many percentages of the pixels are correctly identified by the model when evaluated against some unseen examples. 

### Mean Intersection-over-union (IoU)

This is a more suitable metric for the semantic segmentation task. The IoU score for each class is defined as follows:

$$\frac{TruePositive}{TruePositive + FalsePositive + FalseNegative}$$

and the mean IoU is the average of this score across all classes. The Mean IoU is more suitable because it considers also the false negatives.

## The Dataset

The Dataset is a collection of satellite images for 22 cities across Western Europe and the US. The entire dataset contains 4314 images, collected from Google Satellite Images. Google has a fixed interval of updates to satellite images between 1 to 3 years, so the images in the dataset are all captured from the 2017-2020 period. This means that the nature of the dataset is cross-sectional.

A 8:1:1 train/dev/test split is conducted on the dataset. This results in a training set of 3450 images, dev set of 433 images and 431 images. Validation and fine-tuning of hyperparameters are conducted on the dev set, whereas the test set is used to compute the test accuracy and the mean IoU score. As I extend the dataset to cover more cities outside of the Western hemisphere, the training set could take on a higher weight, to something akin to 9:0.5:0.5.



# Model Training

Below presents the code for model training. To save space, the python code are all stored in separate .py scripts and this Jupyter Notebook sources code from them.

We will begin by mounting Google Drive. This is required for training on Google Colab. If you wish to train offline, you can skip this step.

In [ ]:
from google.colab import drive # for Google Drive Mounting
home = "/content/gdrive/My Drive/Metropolitan Area Dataset"
drive.mount('/content/gdrive')

Mounted at /content/gdrive


And loading a few of the required libraries, particularly the TensorFlow framework:

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

import tensorflow as tf
import datetime
import os
import re

# Clear any logs from previous runs
#rm -rf ./logs/

## Accessing the Dataset and Scripts

If you wish to run the code on your own Colab, please follow these instructions.

1. Open up this [shared folder link](https://drive.google.com/drive/folders/1m38V-wL2gPonnoeZtQYD3jBfLf0dbBth?usp=sharing) on Google Drive.

2. You will now clone the dataset and scripts into your own Google Drive. Please make sure your Google Drive has free space of at least 3GB to save all images and the scripts. Once in the link, go back to the "Shared With Me" Folder:

<img src="https://drive.google.com/uc?id=1lLtDN8bs6qwMapaPLuoOnli4CC128_Tq" alt="Urban" width="512"/>

3. In the "Shared with Me" folder, select the "Metropolitan Area Dataset" Folder. Right click to get the options menu. Click "Add Shortcut to Drive":

<img src="https://drive.google.com/uc?id=1ePZs9Wkeaq8o2iZStOl-r5TKBTRTUNLz" alt="Urban" width="512"/>

4. Click "My Drive" (or the desired folder), and then click "Add Shortcut".

<img src="https://drive.google.com/uc?id=1TO38lECSiAPRLKTk0mBXpFH-uZU3AbQD" alt="Urban" width="512"/>

5. The Dataset is now cloned to your own Google Drive. You can now mount your own Google Drive, access the cloned dataset and scripts and run them on your Google Colab!


## Setup

(This is a detailed explanation of each of the helper scripts I wrote for the pipeline. If you wish to jump straight to the model inference and results discussion, please consult the `training_demo.ipynb` notebook for a less technical discussion.)

We will begin by mounting my Google Drive, importing the TensorFlow framework, and a few base Python packages.

(If you saved the folder somewhere inside My Drive, note that you need change the `home` variable in the following Code Chunk to the corresponding folder. Let's Say the contents of `Metropolitan Area Dataset` live in `My Drive/Research/Datasets/`, then you need to change `home` variable to: `"/content/gdrive/My Drive/Research/Datasets/Metropolitan Area Dataset"`)

### File Path setup

Please ensure that your dataset is structured as follows:
- Dataset: Within a dataset folder, please put your images into the following folders:
- Training Set, Source Image: `dataset/trainset/images/training`
- Training Set, Segmentation Mask: `dataset/trainset/annotations/training`
- Dev Set, Source Image: `dataset/devset/images/testing`
- Dev Set, Segmentation Mask: `dataset/devset/annotations/testing`
- Test Set, Source Image: `dataset/testset/images/testing`
- Test Set, Segmentation Mask: `dataset/testset/annotations/testing`
- Out-of-sample images: `dataset/preds` , with subfolders named after each of the city, and put all images of the same city in the same folder. For example, `dataset/preds/Barcelona` for all satellite images of Barcelona.

In [ ]:
import os
import math
home = "/content/gdrive/My Drive/Metropolitan Area Dataset"
script_path = home
dataset_path = home
training_data = "/trainset/images/training/*"
val_data = "/devset/images/testing/*"
test_data = "/testset/images/testing/*"
checkpoint_dir = dataset_path+"/weights"

### TensorFlow Setup

This section explains each step in `tf_setup.py`.

We will first setup TensorFlow (tf). We begin by printing the version of tf and make sure that tf is at least of version 2.0. There are some parts of the code using the `@tf.function` callback that is only supported after tf 2.0.


In [ ]:
from glob import glob

import IPython.display as display
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import datetime, os
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU
from IPython.display import clear_output


AUTOTUNE = tf.data.experimental.AUTOTUNE
print(f"Tensorflow ver. {tf.__version__}")

SEED = 42



Tensorflow ver. 2.6.0


Then, we check for any available GPUs. On Google Colab, the free account will not be allocated any GPUs. On Google Colab Pro, you will probably get assigned a Tesla A100 GPU.

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

### File path setting

Here `dataset_path` is aliased to be `script_path` because on my Google Drive they are the same location. You might have to change it if your dataset lives elsewhere.

Next, we specify the size of the image to be cropped. Recall that each image has 3 channels, so increasing the size of the image will impose extra requirements on computational power.

Then, we specify the number of channels and classes. Most images have 3 channels (RGB). Some images (e.g. grayscale) may have one. Therefore, the value here depends on the type of image used.

The number of classes can go over 2. In this model I take advantage of the fact that the binary crossentrophy is a special case of the categorical crossentrophy loss function (more discussion below), so we can set the number of classes to be 2 and use the categorical crossentrophy to predict on the non-building and building classes. If you have more than one type of labels, you can increase this number.

(only for binary classification:) The threshold parameter controls the threshold of coercion for grayscale images. Any numeric values above this threshold will be coerced to a 1, and 0 otherwise. So imagine if we set this threshold to 128, anything that is above the middle of the gray scale will be coerced to a full bright pixel (1), and everything else below would be coerced to a full dark pixel (0).

In [ ]:
# Image size that we are going to use. >=1.
IMG_SIZE = 128
# Number of channels in an image. # either 1 or 3.
N_CHANNELS = 3
# Number of classes. Has to be at least 2.
N_CLASSES = 2

# Threshold to determine if something is a building (luminosity):
threshold = 128

TRAINSET_SIZE = len(glob(dataset_path + training_data + "*.jpg"))
print(f"The Training Dataset contains {TRAINSET_SIZE} images.")

VALSET_SIZE = len(glob(dataset_path + val_data + "*.jpg"))
print(f"The Validation Dataset contains {VALSET_SIZE} images.")

TESTSET_SIZE = len(glob(dataset_path + test_data  + "*.jpg"))
print(f"The Testing Dataset contains {TESTSET_SIZE} images.")

exec(open(os.path.join(script_path + "/data_loader.py")).read())

The Training Dataset contains 3450 images.
The Validation Dataset contains 433 images.
The Testing Dataset contains 431 images.


## Data Pipeline

This is a detailed explanation of the helper functions defined in `data.loader.py`.

### parse_img()

This is the image parser that every data loader would use. It loads an image, decodes it, and then find the associated mask of that image. The mask is then coerced to an array of 0 and 1 classes based on `threshold`. The output is a dictionary containing the path to the image and the path to its segmentation mask.

In [ ]:
def parse_image(img_path: str,  threshold: int) -> dict:
    """Load an image and its annotation (mask) and returning
    a dictionary.

    Parameters
    ----------
    img_path : str
        Image (not the mask) location.

    threshold: int
        threshold used to coerce pixels to a 0 and 1 class. Anything
        above the threshold will be coerced to 1 and 0 otherwise.

    Returns
    -------
    dict
        Dictionary mapping an image and its annotation.
    """
    image = tf.io.read_file(img_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.uint8)

    # For one Image path:
    # .../trainset/images/training/ADE_train_00000001.jpg
    # Its corresponding annotation path is:
    # .../trainset/annotations/training/ADE_train_00000001.png
    mask_path = tf.strings.regex_replace(img_path, "images", "annotations")
    #mask_path = tf.strings.regex_replace(mask_path, "jpg", "png")
    mask = tf.io.read_file(mask_path)
    # The masks contain a class index for each pixels
    ############ change file type here ##################
    mask = tf.image.decode_jpeg(mask, channels=1)

    ############ Coerce mask to two types of classes
    mask = tf.where(mask <= threshold, np.dtype('uint8').type(0), mask)
    mask = tf.where(mask != 0 , np.dtype('uint8').type(1), mask)

    # Note that we have to convert the new value (0)
    # With the same dtype than the tensor itself

    return {'image': image, 'segmentation_mask': mask}

### normalize()

This is a simple helper function that normalizes an image from the 0-255 scale to 0-1 scale.

In [ ]:
@tf.function
def normalize(input_image: tf.Tensor, input_mask: tf.Tensor) -> tuple:
    """Rescale the pixel values of the images between 0.0 and 1.0
    compared to [0,255] originally.

    Parameters
    ----------
    input_image : tf.Tensor
        Tensorflow tensor containing an image of size [SIZE,SIZE,3].
    input_mask : tf.Tensor
        Tensorflow tensor containing an annotation of size [SIZE,SIZE,1].

    Returns
    -------
    tuple
        Normalized image and its annotation.
    """
    input_image = tf.cast(input_image, tf.float32) / 255.0
    return input_image, input_mask

### load_image_train()

This function prepares the training set image. It resizes the image, then applies a right rotation randomly. After that, it normalizes the output image.

In [ ]:
@tf.function 
def load_image_train(datapoint: dict) -> tuple:
    """Apply some transformations to an input dictionary
    containing a train image and its annotation.

    Notes
    -----
    An annotation is a regular  channel image.
    If a transformation such as rotation is applied to the image,
    the same transformation has to be applied on the annotation also.

    Parameters
    ----------
    datapoint : dict
        A dict containing an image and its annotation.

    Returns
    -------
    tuple
        A modified image and its annotation.
    """
    input_image = tf.image.resize(datapoint['image'], (IMG_SIZE, IMG_SIZE))
    input_mask = tf.image.resize(datapoint['segmentation_mask'], (IMG_SIZE, IMG_SIZE))

    draw = tf.random.uniform(shape=[])

    input_image = tf.cond(tf.math.greater(draw, 0.5),
        lambda: tf.image.flip_left_right(input_image),
        lambda: input_image)

    input_mask = tf.cond(tf.math.greater(draw, 0.5),
        lambda: tf.image.flip_left_right(input_mask),
        lambda: input_mask)

    # Random rotation - how to not use operator???!?!?!?!?
    #if tf.math.greater(tf.random.uniform(shape=[]), 0.5) :
    #    input_image = tf.image.flip_left_right(input_image)
    #    input_mask = tf.image.flip_left_right(input_mask)

    input_image, input_mask = normalize(input_image, input_mask)

    return input_image, input_mask

The following script summarizes the training environment (Nr. CPUs and Nr. GPUs),  sizes of the train/dev/test sets, and a random sample of the image and its segmentation mask.

Note that I have sized all images to $128*128$ to save computation cost. After all, parsing one $128*128*3 = 49152$ digits is already quite hefty on the machine!

Randomly initialized weights would be used to predict the segmentation mask. The intialized weights should be a very poor prediction of the segmentation mask.

### load_image_test()

This is the same data loader pipeline as `load_image_train()` except that it omits the random right rotation segment. When testing we do not need to randomly rotate the image, we use the image as is to check for testing accuracy.

In [ ]:
@tf.function
def load_image_test(datapoint: dict) -> tuple:
    """Normalize and resize a test image and its annotation.

    Notes
    -----
    Since this is for the test set, we don't need to apply
    any data augmentation technique.

    Parameters
    ----------
    datapoint : dict
        A dict containing an image and its annotation.

    Returns
    -------
    tuple
        A modified image and its annotation.
    """

    input_image = tf.image.resize(datapoint['image'], (IMG_SIZE, IMG_SIZE))
    input_mask = tf.image.resize(datapoint['segmentation_mask'], (IMG_SIZE, IMG_SIZE))

    input_image, input_mask = normalize(input_image, input_mask)

    return input_image, input_mask

### parse_img_pred()

This function parses an out-of-sample image for use in model prediction. It is similar to `load_image_test()` except that the image has no segmentation mask to operate on (since this is for out of sample cases).

In [ ]:
@tf.function
def parse_image_pred(img_path: str) -> float:
    """Normalize and resize a test image and its annotation.

    Notes
    -----
    Since this is for the test set, we don't need to apply
    any data augmentation technique.

    Parameters
    ----------
    datapoint : dict
        A dict containing an image and its annotation.

    Returns
    -------
    tuple
        A modified image and its annotation.
    """

    """Load an image and its annotation (mask) and returning
    a dictionary.

    Parameters
    ----------
    img_path : str
        Image (not the mask) location.

    Returns
    -------
    dict
        Dictionary mapping an image and its annotation.
    """
    image = tf.io.read_file(img_path)
    image = tf.image.decode_jpeg(image, channels=3)

    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))

    image = tf.cast(image, tf.float32) / 255.0

    return image

We will now initialize some weights, and generate predictions to see if the model is producing output in the expected dimensions:

## Model Building

This section explains the model building process in the `model_compile.py` helper script.

## Software Requirements

If you wish to run the script locally, please ensure that your local machine fulfills the following software requirements:

- TensorFlow version >=2.6
- Python version >=3.7

### Visualization Tools

Below show several visualization helper functions defined to help us fetch images and visualize the model output.

- display_sample() is used to plot an example from the dataset. takes a dictionary with the `image` and `segmentation_mask` paths, and plot them side by side.
- create_mask() is used to convert the predicted segmentation mask into an image ready to plot by finding the class with the highest probability for each class. This is more important for multi-class segmentation problems.
- show_predictions(): This function plots the source image, ground truth segmentation mask and the model prediction side by side. When a dataset is not defined, it will randomly draw one example from the training set.

In [ ]:
def display_sample(display_list):
    """Show side-by-side an input image,
    the ground truth and the prediction.
    """
    plt.figure(figsize=(18, 18))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()


def create_mask(pred_mask: tf.Tensor) -> tf.Tensor:
    """Return a filter mask with the top 1 predictions
    only.

    Parameters
    ----------
    pred_mask : tf.Tensor
        A [IMG_SIZE, IMG_SIZE, N_CLASS] tensor. For each pixel we have
        N_CLASS values (vector) which represents the probability of the pixel
        being these classes. Example: A pixel with the vector [0.0, 0.0, 1.0]
        has been predicted class 2 with a probability of 100%.

    Returns
    -------
    tf.Tensor
        A [IMG_SIZE, IMG_SIZE, 1] mask with top 1 predictions
        for each pixels.
    """
    # pred_mask -> [IMG_SIZE, SIZE, N_CLASS]
    # 1 prediction for each class but we want the highest score only
    # so we use argmax
    pred_mask = tf.argmax(pred_mask, axis=-1)
    # pred_mask becomes [IMG_SIZE, IMG_SIZE]
    # but matplotlib needs [IMG_SIZE, IMG_SIZE, 1]
    pred_mask = tf.expand_dims(pred_mask, axis=-1)
    return pred_mask

def show_predictions(dataset=None, num=1, num_in_batch=0):
    """Show a sample prediction.

    Parameters
    ----------
    dataset : [type], optional
        [Input dataset, by default None
    num : int, optional
        Number of batches to show, by default 1
    num_in_batch :  int, optional
        The index number of samples within a batch to show.
    """

    ### If there's a dataset, this would sample one batch.
    if dataset:
        for image, mask in dataset.take(num):
            pred_mask = model.predict(image)
            display_sample([image[num_in_batch], mask[num_in_batch], create_mask(pred_mask[num_in_batch])])
    else:
        # The model is expecting a tensor of the size
        # [BATCH_SIZE, IMG_SIZE, IMG_SIZE, 3]
        # but sample_image[0] is [IMG_SIZE, IMG_SIZE, 3]
        # and we want only 1 inference to be faster
        # so we add an additional dimension [1, IMG_SIZE, IMG_SIZE, 3]
        one_img_batch = sample_image[0][tf.newaxis, ...]
        # one_img_batch -> [1, IMG_SIZE, IMG_SIZE, 3]
        inference = model.predict(one_img_batch)
        # inference -> [1, IMG_SIZE, IMG_SIZE, N_CLASS]
        pred_mask = create_mask(inference)
        # pred_mask -> [1, IMG_SIZE, IMG_SIZE, 1]
        display_sample([sample_image[0], sample_mask[0],
                        pred_mask[0]])
                        
# for image, mask in dataset['train'].take(1):
#     sample_image, sample_mask = image, mask

#show_predictions()

### Train/Dev/Test Parsing

We will now use the helper functions from the data pipeline to parse the images in the training, dev and test sets. Note that the train/dev/test split is not conducted in this file. Train/Test split is only conducted once at the file-level.

Note that the train and dev sets are all constructed as infinite samples that can be drawn from using `.repeat()`. Therefore, the user would have to manually specify how many `EPOCHS` to draw from, and how many images are in one minibatch (`BATCH_SIZE`).

**Failure to define these parameters would result in the data loader stuck in an infinite loop**

In [ ]:
train_dataset = tf.data.Dataset.list_files(dataset_path + training_data, seed=SEED, shuffle=False)

train_dataset = train_dataset.map(lambda x: parse_image(x, threshold))

val_dataset = tf.data.Dataset.list_files(dataset_path + val_data + "*.jpg", seed=SEED, shuffle=False)
val_dataset = val_dataset.map(lambda x: parse_image(x, threshold))

test_dataset = tf.data.Dataset.list_files(dataset_path + test_data, seed=SEED, shuffle=False)

test_dataset = test_dataset.map(lambda x: parse_image(x, threshold))

BUFFER_SIZE = 1000
BATCH_SIZE = 32

assert BATCH_SIZE >0

dataset = {"train": train_dataset, "val": val_dataset, "test": test_dataset}

# -- Train Dataset --#
dataset['train'] = dataset['train'].map(load_image_train)
dataset['train'] = dataset['train'].shuffle(buffer_size=BUFFER_SIZE, seed=SEED)
dataset['train'] = dataset['train'].repeat()
dataset['train'] = dataset['train'].batch(BATCH_SIZE)
dataset['train'] = dataset['train'].prefetch(buffer_size=AUTOTUNE)

#-- Validation Dataset --#
dataset['val'] = dataset['val'].map(load_image_test)
dataset['val'] = dataset['val'].repeat()
dataset['val'] = dataset['val'].batch(BATCH_SIZE)
dataset['val'] = dataset['val'].prefetch(buffer_size=AUTOTUNE)

#-- Test Dataset --#
dataset['test'] = dataset['test'].map(load_image_test)
#dataset['test'] = dataset['test'].repeat()
dataset['test'] = dataset['test'].batch(BATCH_SIZE)
dataset['test'] = dataset['test'].prefetch(buffer_size=AUTOTUNE)


### Defining the U-Net Architecture

This section defines the entire U-Net Architecture. Note that parameters are intialized using the `he_normal` (He Initializer). The main reason for choosing the He Intializer is that it works better with ReLU activations. For a detailed explanation, see [This StackExchange Discussion](https://stats.stackexchange.com/questions/319323/whats-the-difference-between-variance-scaling-initializer-and-xavier-initialize/319849#319849).

In [ ]:
# -- Keras Functional API -- #
# -- UNet Implementation -- #
# Everything here is from tensorflow.keras.layers
# I imported tensorflow.keras.layers * to make it easier to read
dropout_rate = 0.5
input_size = (IMG_SIZE, IMG_SIZE, N_CHANNELS)

# If you want to know more about why we are using `he_normal`:
# https://stats.stackexchange.com/questions/319323/whats-the-difference-between-variance-scaling-initializer-and-xavier-initialize/319849#319849  
# Or the excellent fastai course:
# https://github.com/fastai/course-v3/blob/master/nbs/dl2/02b_initializing.ipynb
initializer = 'he_normal'


# -- Encoder -- #
# Block encoder 1
inputs = Input(shape=input_size)
conv_enc_1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer=initializer)(inputs)
conv_enc_1 = Conv2D(64, 3, activation = 'relu', padding='same', kernel_initializer=initializer)(conv_enc_1)

# Block encoder 2
max_pool_enc_2 = MaxPooling2D(pool_size=(2, 2))(conv_enc_1)
conv_enc_2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(max_pool_enc_2)
conv_enc_2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(conv_enc_2)

# Block  encoder 3
max_pool_enc_3 = MaxPooling2D(pool_size=(2, 2))(conv_enc_2)
conv_enc_3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(max_pool_enc_3)
conv_enc_3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(conv_enc_3)

# Block  encoder 4
max_pool_enc_4 = MaxPooling2D(pool_size=(2, 2))(conv_enc_3)
conv_enc_4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(max_pool_enc_4)
conv_enc_4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(conv_enc_4)
# -- Encoder -- #

# ----------- #
maxpool = MaxPooling2D(pool_size=(2, 2))(conv_enc_4)
conv = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(maxpool)
conv = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(conv)
# ----------- #

# -- Decoder -- #
# Block decoder 1
up_dec_1 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = initializer)(UpSampling2D(size = (2,2))(conv))
merge_dec_1 = concatenate([conv_enc_4, up_dec_1], axis = 3)
conv_dec_1 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(merge_dec_1)
conv_dec_1 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(conv_dec_1)

# Block decoder 2
up_dec_2 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = initializer)(UpSampling2D(size = (2,2))(conv_dec_1))
merge_dec_2 = concatenate([conv_enc_3, up_dec_2], axis = 3)
conv_dec_2 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(merge_dec_2)
conv_dec_2 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(conv_dec_2)

# Block decoder 3
up_dec_3 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = initializer)(UpSampling2D(size = (2,2))(conv_dec_2))
merge_dec_3 = concatenate([conv_enc_2, up_dec_3], axis = 3)
conv_dec_3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(merge_dec_3)
conv_dec_3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(conv_dec_3)

# Block decoder 4
up_dec_4 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = initializer)(UpSampling2D(size = (2,2))(conv_dec_3))
merge_dec_4 = concatenate([conv_enc_1, up_dec_4], axis = 3)
conv_dec_4 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(merge_dec_4)
conv_dec_4 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(conv_dec_4)
conv_dec_4 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(conv_dec_4)
# -- Dencoder -- #

output = Conv2D(N_CLASSES, 1, activation = 'softmax')(conv_dec_4)

## Metrics

The MeanIoU has to be manually updated for use with the Sparse Categorical CrossEntropy.

In [ ]:
#MeanIou For use with the Sparse Categorical Cross Entrophy 
class UpdatedMeanIoU(tf.keras.metrics.MeanIoU):
  def __init__(self,
               y_true=None,
               y_pred=None,
               num_classes=None,
               name=None,
               dtype=None):
    super(UpdatedMeanIoU, self).__init__(num_classes = num_classes,name=name, dtype=dtype)

## Model Compilation

The following scripts compile the model. As discussed above, I'm using a learning rate decay with rate 0.9 and I have set a ceiling of decay to 10000 steps.

A note about the loss function: While our ML problem is binary classification, here I used the Sparse Categorical Crossentropy. This is to take advantage of the fact that the Binary Crossentropy is a special case of Categorical Crossentropy when the number of classes is exactly at 2:

$$CCE = L(\theta) = -\frac{1}{n} \sum^n_{i=1}\sum^m_{j=1}y_{ij} log(p_{ij})$$
$$= -\frac{1}{n}\sum^n_{i=1} [y_i log(p_i) + (1-y_i) log(1-p_i)] = BCE$$

If I keep using the Categorical Crossentropy I don't have to edit the compilation code below for a different ML problem. But the downside is that the prediction scripts will have to be tweaked for a multiclass classification problem because the dimensions of the output tensors will be different than the case for binary classification.

The only downside of this approach is that when dealing with the output tensor, they will have 2 channels (for the two classes - 0s and 1s), so to get the number of pixels predicted to be a 1 I need to use `tf.reshape` to coerce the second channel (for the 1s) to be a single channel tensor from the model output. For a multiclass problem, we can use the `argmax` across channels to create an output image where the pixels are all chosen from the classes with the highest predicted probability.

In [ ]:
############################################# COMPILE MODEL ###################################################

# LR Decay "Schedule" - can be replaced with the learning_rate parameter.
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)

def update_state(self, y_true, y_pred, sample_weight=None):
  y_pred = tf.math.argmax(y_pred, axis=-1)
  return super().update_state(y_true, y_pred, sample_weight)

model = tf.keras.Model(inputs = inputs, outputs = output)
model.compile(optimizer=Adam(learning_rate=lr_schedule), loss = tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy',UpdatedMeanIoU(num_classes=2)])

## Mini-batching Settings

The one final step before we proceed with the training is the mini-batching settings. Note that `EPOCHS` must be manually defined, **otherwise the data loader will be stuck in an infinite loop**. An assertion is already defined to prevent this.

In [ ]:
######################### Mini-batching settings ######################

STEPS_PER_EPOCH = TRAINSET_SIZE // BATCH_SIZE

# Using #dataset.repeat to randomly sample some 50 epochs and train on them.
EPOCHS = 30

VALIDATION_STEPS = VALSET_SIZE // BATCH_SIZE

assert EPOCHS>=1

## Testing intialized weights

Without training, if we call `show_predictions()` it will generate predictions from initialized weights. We will expect a poorly predicted mask from the model. This is to check if the model is compiled correctly, and if it generates the segmentation mask in the expected dimensions.

In [ ]:
show_predictions()

The intialized weights are all over the place, and thus the prediction is obviously all off. All is good.

## Model Summary

Below defines a summary of the entire model.

In [ ]:
model.summary()

Just like the U-Net architecture states, in the encoding stage the convolutions are applied increasingly more and more, so you see that the number of channels increase while the dimension of the input image shrinks. After each series of convolutions, a skip connection is made with `Concatenate()`. Later, once we are in the decoding stage, we perform convolutions such that the number of channels decrease and slowly make it back to the original image dimensions. In each decoding stage we will use the cached output from the encoding stage.

The model has a total of 31 million parameters to be learned! This is why fitting this model can take a long time. Accelerating the computation with a graphics card is almost necessary for this task. Using the Nvidia GTX 1060 on my local machine, I was able to pass through one epoch of 93 images in around 5 minutes.

## Model Training

We will now begin training the model. It takes a while without GPU acceleration. On my local machine using a Nvidia GTX 1060 Graphics card, one epoch of 107 images can be processed in around a minute.

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# On GPU
model_history = model.fit(dataset['train'], epochs=EPOCHS,
                          steps_per_epoch=STEPS_PER_EPOCH,
                          validation_steps=VALIDATION_STEPS,
                          validation_data=dataset['val'],
                          callbacks = [tensorboard_callback])

%tensorboard --logdir logs/fit

## Loading pre-trained weights

To save the training time, I pre-fitted the weights on my local machine. There are three sets of weights:

- Day 2 - Final weights trained with $\alpha$ fixed at .0001.
- Day 2 - LRD trained with $\alpha$ subject to learning rate decay, rate=0.9 for the first 10000 iterations.

I will now load these pre-trained weights to avoid training the model on Google Colab.

In [ ]:
checkpoint_path = dataset_path+"/weights/"+"cp-day2-lrd.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

## Check Test Accuracy

When tested against the test set, the accuracy and Mean IoU scores are computed. The model with learning rate decay has the best performance. With test set accuracy  at 90.99% and the updated mean IoU score at 90.47%.


In [ ]:
model.load_weights(checkpoint_dir+"/cp-day2.ckpt")

loss, acc, miou = model.evaluate(dataset['test'], verbose=2)
print("Day 2 Model, accuracy: {:5.2f}%".format(100 * acc), "Mean IoU: {:5.2f}%".format(100 * miou), sep=" ")

model.load_weights(checkpoint_dir+"/cp-day2-lrd.ckpt")

loss, acc, miou = model.evaluate(dataset['test'], verbose=2)
print("Day 2 Model (+Learning Rate Decay), accuracy: {:5.2f}%".format(100 * acc), "Mean IoU: {:5.2f}%".format(100 * miou), sep=" ")


## Example Prediction

We can take a test set example and visualize the model predictions:

In [ ]:
model.load_weights(checkpoint_dir+"/cp-day2-lrd.ckpt")
draw = int(np.random.randint(low=0, high=BATCH_SIZE-1, size=1)) #randomly draw a number between 0-32

show_predictions(dataset=dataset['test'],num=1, num_in_batch = draw)

# Constructing the Index

The index for a particular area can be conducted by parsing all satellite images of a city through the model. The predictions for each image will then be a 128*128 float point array (due to the use of the softmax activation), where a pixel of 0.0 means that the pixel is not a building (completely black), and a 1.0 means that the pixel is a building with 100% probability.

If we sum across all images $g$: $\sum^g_{i=1}p_i$, this is a rough measure of the total level development of a city since it is a sum of all building pixels across all images.

However, this index must be weighted because the geographical size of each city is different. In some very large cities like Beijing or Moscow this measure is likely to be very large due to the sheer size of images parsed through the model.

To acquire a sense of the **per capita** development, this measure is weighted by the population of a city.

To get a sense of how unequal/ spread out is the development, we can either compute a correlation of variation (CV) or a gini coefficient. For the correlation of variation weighted by population, this is simply:

$$CV_{weighted} = 100*(\frac{SD}{Mean})*\frac{1}{Population}$$

This essentially means the per capita percentage of deviation of the index from its mean. Note that the mean is calculated at the image level. The number of images is dependent on the size of the city. But then we weight that again with the population, so we essentially have controlled for both city size and population size.

The gini coefficient can be defined as the following:
$$G = \frac{\sum^n_{i=1}\sum^n_{j=1}|x_i - x_j|}{2n^2\bar{x}}$$

This alternative definition takes advantage of the fact that the Gini coefficient is half of the relative mean absolute difference, a mathematically equivalent definition of the Lorenz curve. 

Note that the gini coefficient can only be computed at the image level. Because our measurement of "wealth" is at the image level, any method to compute the Gini Coefficient would always return a measure of inequality at the image level, not the population level. Therefore, the Gini Coefficient is computed in the dataset but not used in the main analysis.

The functions to compute the index is provided below:

In [ ]:
import pandas as pd #required for manipulation of the data as a dataset.

def gini(x: pd.Series) -> float:
    '''
    A **bruteforce** function to compute the gini coefficient.
    It computes the MAD of a distribution, and then reweights it with the relative MAD.

    Input: x: pd.Series - a pandas series representing a distribution.
    
    Output: g-> float : A float point value between -1 and 1 representing the degree of income inequality.
    -1 means absolutely no inequality and 1 means full inequality.

    Time Complexity: O(n**2)
    Space Complexity: O(n**2)

    Do NOT use for very large samples, e.g. n>10000.
    '''
    x = x.to_numpy(dtype=np.float32)

    x = x[np.logical_not(np.isnan(x))]
    # Mean absolute difference
    mad = np.abs(np.subtract.outer(x, x)).mean()
    # Relative mean absolute difference
    rmad = mad/np.mean(x)
    # Gini coefficient
    g = 0.5 * rmad
    return g

def makeDataset(val: pd.Series) -> pd.DataFrame:
    '''
    Description: Make a dataset from a list of named lists.

    Input: val: pd.Series
      pd.Series of predictions for a city, with the name of the series set to the name of that city.

    Output: pd.DataFrame
      A pd.DataFrame containing all predictions for a city as columns.
    '''
    df = {}
    for city,value in zip(ref,val):
        df['city'] = val
    return df

def acquireValues(path: str, normalized: bool) -> pd.Series:
    '''
    Description: a Function to translate satellite images from a folder to a measure of development.

    Inputs:
    - path (str) : The path to the folder containing .jpg satellite images.
    - normalized (bool): Should the program return normalized predictions or unnormalized image predictions?
      Normalization is defined as the coercion of pixels from scale 0 - 255 (RGB) to 0-1.
      Note that if normalized, then the images cannot be displayed properly by keras.utils.array_to_img().

    Outputs:
    - A pd.Series containing the sum of bright pixels in an image.
    - pd.Series.name - Contains the name of the folder (assumed to be name of city)
    '''
    ### the following code is for windows where using os.path.join would result in a unique identifier \\
    # for one path
    #city = re.search(r'\\([a-zA-Z]+$)',path)
    #city = city.group(1) #Acquire Name of city via Regex

    #for linux/mac, use this:
    city = os.path.basename(os.path.normpath(path)) #normpath gives us the "normalized" path by removing anything in between folders. #basename gives the current folder name.

    print("|","Processing:",city, sep=" ")

    TESTSET_SIZE = len(glob(path+ "/*.jpg"))
    print("|","City:",city,"contains",TESTSET_SIZE,"images", sep=" ")

    #correcting path
    path = tf.strings.regex_replace(path, "\\\\", "/")

    data_for_pred = tf.data.Dataset.list_files(path+"/*.jpg", seed=SEED, shuffle=False)
    data_for_pred = data_for_pred.map(parse_image_pred)
    data_for_pred = data_for_pred.batch(BATCH_SIZE)
    data_for_pred = data_for_pred.prefetch(buffer_size=AUTOTUNE)

    print("|--->","City",city,"has been processed by tf.data.Dataset.", sep =" ")

    preds = model.predict(data_for_pred)

    print("|--->","Predictions for",city,"acquired", sep=" ")

    if normalized == True:
        preds = tf.reshape(preds[:,:,:,1], (TESTSET_SIZE,IMG_SIZE, IMG_SIZE))
    else:
        preds = tf.reshape(preds[:,:,:,1]*255.0, (TESTSET_SIZE,IMG_SIZE, IMG_SIZE))

    preds = tf.math.reduce_sum(preds, axis=[1,2])

    out = pd.Series(list(preds), dtype=np.float32, name=city)

    print("|--->","Predictions for",city,"restructured to pd.Series.", sep=" ")

    return out

## Set up

Let's specify the location of the prediction dataset. This script would take each city as its own test set, parse all images for each city and generate predictions:

In [ ]:
pred_dataset = "/preds"
#reload the data loader for updated functions
exec(open(os.path.join(script_path + "/data_loader.py")).read())

In [ ]:
cities = [] #initialize a list of folders containing images for each city.

for root,dirs,files in os.walk(dataset_path + pred_dataset):
    for x in dirs:
        if(re.match(r'[a-zA-Z]',x)):
            cities.append(dataset_path + os.path.join(pred_dataset, x)) #using os.walk to find all folders

print(cities)

data = [acquireValues(x,True) for x in cities] #With a list comprehension, acquire a 2d array of predictions.
# The 2d array will show all the predictions for each city (in columns)

ds = pd.DataFrame(data=data) #convert  to data frame.
ds = ds.T # transpose the output.

In [ ]:
sums = ds.sum(axis=0) #get the sum
means = ds.mean(axis=0) #get the mean
sds = ds.std(axis=0) #get the stddev

ginis = [gini(ds.iloc[:,x]) for x in range(len(ds.columns))] #get gini
ginis = pd.Series(ginis, name="Gini Coefficient")
ginis.index = means.index

ds_stats = pd.DataFrame({"sum":sums, "mean":means, "sd":sds, "gini":ginis}) #put into a ds

ds_stats.head(3)

#ds_stats.to_csv(script_path+"/metro_img_stats.csv") #save to csv. Population data has to be merged from another dataset using SQL join. That part is done in R.

# Voilà!

This concludes the walkthrough of the data pipeline, from image preprocessing to index construction for this project. If you have any questions, please do not hesitate to contact me at my work email: kylechan@unc.edu.
